# Buchbergers Algorithm

## Boiler Plate

First load the Sympy module

In [2]:
## Sympy Boiler Plate

from __future__ import division
from sympy import *
t, x, y, z = symbols('t x y z')  # I believe this order matters, i.e. t<-x1, x <- x2, y <- x3 ...
k, m, n = symbols('k m n', integer=True)
f, g, h = symbols('f g h', cls=Function)
init_printing()
init_printing(use_latex='mathjax', latex_mode='equation')



import pyperclip
def lx(expr):
    pyperclip.copy(latex(expr))
    print(expr)
# NB: See also srepr(), display() and print_mathml()

import numpy as np
import matplotlib as plt

## Terminology

There are mutlidegree's, leading coefficients, leading monomials and leading terms.

In [3]:
def f(x, domain = 'QQ'):
    return 2+3*x+5*y+7*x*y+11*x**2*y+13*x*y**2+17*x**2*y**2+19*x**2*y**2*z**2
    # return 13*x**2+11*x-7
def g(x, domain = 'QQ'):
    return 2+3*x+5*y+7*x*y+11*x**2*y+13*x*y**2+17*x**2*y**2+19*x*y**2*z**2
    # return 13*x**2+11*x-7


In [4]:
print("Leading Monomial:")
display(LM(f(x)))

print("Leading Coefficient")
display(LC(f(x)))

print("Leading Term")
display(LT(f(x)))

Leading Monomial:


 2  2  2
x ⋅y ⋅z 

Leading Coefficient


19

Leading Term


    2  2  2
19⋅x ⋅y ⋅z 

The initial monomial with respect to an ordering is the first monomial returned by `sympy`, I believe that sympy uses lexicographic ordering under the hood, as suggested by the documentation [^spdocsPolySysRef], but I'd need to get a better understanding of the differences between `lex`, `grlex` and `grevlex` to be certain.


[^spdocsPolySysRef]: SymPy Development Team. “Polynomials Manipulation Module Reference — SymPy 1.8 Documentation.” Sympy Documentation. Accessed April 13, 2021. https://docs.sympy.org/latest/modules/polys/reference.html.

In [5]:
f(x)

    2  2  2       2  2       2           2                        
19⋅x ⋅y ⋅z  + 17⋅x ⋅y  + 11⋅x ⋅y + 13⋅x⋅y  + 7⋅x⋅y + 3⋅x + 5⋅y + 2

In this case, the first monomial is $x^2y^2z^2$ and so that is the initial. In practice we can just use the `LT`, the initial is just another way to skin the cat, which is probably why sympy doesn't offer a clear way to extract the initial. [^initVlt].


[^initVlt]: If it was necessary to get the initial one option might be to extract the polynomial to a string `str(f(x))` and then use regex `^[A-Za-z0-9*]+(?=[\ +])` to extract the first term and then take the `LM` of that term to get the initial monomial.

In [6]:
f(x)

    2  2  2       2  2       2           2                        
19⋅x ⋅y ⋅z  + 17⋅x ⋅y  + 11⋅x ⋅y + 13⋅x⋅y  + 7⋅x⋅y + 3⋅x + 5⋅y + 2

In [7]:
g(x)

    2  2       2           2  2         2                        
17⋅x ⋅y  + 11⋅x ⋅y + 19⋅x⋅y ⋅z  + 13⋅x⋅y  + 7⋅x⋅y + 3⋅x + 5⋅y + 2

## Implementing the Algorithm

### Set of Polynomials

To begin with, we'll implement buchberger's algorithm on a simple polynomial system, the reason for this choice is to see the implementation of the algorithm generally, a *simple* system such as a linear or single variable system, may be more confusing at first, but will be returned to later in order to draw insights.

So we have our bag of polynomials:

In [8]:
F = [x**3-2*x*y, x**2*y-2*y**2 + x]
for poly in F:
    display(poly)

 3        
x  - 2⋅x⋅y

 2            2
x ⋅y + x - 2⋅y 

These polynomials are equal to zero and so could also be expressed:

\begin{align}
x^3 &= 2xy \\
  x &=2y^2-x^2y    
\end{align}

Now we want our algorithm to give us back:

In [9]:
G_out = polys.polytools.groebner(F, x, y, order = 'lex', method = 'buchberger')
G_out


             ⎛⎡       2   3⎤                           ⎞
GroebnerBasis⎝⎣x - 2⋅y , y ⎦, x, y, domain=ℤ, order=lex⎠

Note that the 2nd tupple $\left(x, y\right)$ that is printed out refers to the variables, the first tupple is the reduced Groebner Basis, a clearer way to display it would be as a list:

In [10]:

G = []
for poly in G_out:
    G.append(poly)

display(G)


⎡       2   3⎤
⎣x - 2⋅y , y ⎦

and so overall we have:

$$
\begin{array}{rcl}
x^{3} & = & 2xy\\
x & = & 2y^{2}-x^{2}y
\end{array}\implies\begin{array}{rcl}
y^{2} & = & \frac{1}{2}x\\
y^{3} & = & 0
\end{array}
$$

\begin{align}
y^2 &= \frac{1}{2}x \\
y^3 &= 0
\end{align}    

## S-Polynomial

The $S$-Polynomial used to determine whether or not two polynomials belong in the Groebner Basis and it is given by:

$$
S=\mathrm{LCM}\left(\mathrm{LM}\left(f\right),\mathrm{LM}\left(g\right)\right)\times\left(\frac{f}{\mathrm{LT}\left(f\right)}-\frac{g}{\mathrm{LT}\left(g\right)}\right)
$$

Sympy comes with a function to determine the LCM of polynomials:



In [11]:
from sympy.polys.monomials import monomial_mul, monomial_lcm, monomial_divides, term_div
# https://docs.sympy.org/latest/modules/polys/basics.html
LMf = LM(F[0])
LMg = LM(F[1])
# LCM12 = monomial_lcm(LMF, LMG) # This fails
LCM12 = lcm(LMf, LMg)
LCM12


 3  
x ⋅y

and the $s$-polynomial can hence be calculated thusly:

In [12]:
def s_polynomial(f, g):
    LCM_fg = lcm(LM(f), LM(g))
    s = LCM_fg*(f/LT(f)-g/LT(g))
    return s.expand()
s=s_polynomial(F[0], F[1])
s

  2
-x 

Now we need to divide this $s$ value by all the terms in $F$, write it out in terms of quotients and divisors and then consider the remainder, so for example:

In [410]:
display(div(s, F[0]))
display(div(s, F[1]))

⎛     2⎞
⎝0, -x ⎠

⎛     2⎞
⎝0, -x ⎠

\begin{align*}
 & -x^{2} & = & 0\left(2xy\right) & -x^{2}\\
+ &  & =\\
 & -x^{2} & = & 0\left(2y^{2}-x^{2}y\right) & -x^{2}\\
\hline  & -2x^{2} & = & 0\left(2xy\right)+0\left(2y^{2}-x^{2}y\right) & -2x^{2}\\
\implies & r & = & \mathrm{mean}\left(r\in R\right)
\end{align*}


And so this remainder value can be calculated, in `sympy`, like so:

In [411]:
 def remainder_set_division(s, F):
    """
    return remainder value after dividing s by all terms in F,
    For use in BuchBerger's Algorithm using an S-polynomial against the
    collection of polynomials

    Args:
        s (sympy): A sympy polynomial, typically an S-polynomial for 
        F (list, of sympy): The list of polynomials of concern
    """    
    r_list = [ div(s, F[i])[1] for i in range(len(F)) ]
    r = sum(r_list)/len(r_list)
    return r

r = remainder_set_division(s, F)    
r

  2
-x 

Now because this remainder value is non-zero, it belongs to the Groebner Bases and it needs to be added back to $F$ and the process needs to start again.

In [412]:
F_0 = F.copy() # Remember to copy lists in Python, like C
F.append(r)
for poly in F:
    display(poly)


 3        
x  - 2⋅x⋅y

 2            2
x ⋅y + x - 2⋅y 

  2
-x 

### Continue testing values

Next compare $f_1$ and the newly added $f_3$:

In [413]:
# Calculate the S-Polynomial
s = s_polynomial(F[0], F[2])
print("The s polynomial is:")
display(s)
# calculate the remainder
r = remainder_set_division(s, F)    
print("The remainder is:")
display(r)

# Add the remainder back to F
if r!=0:
    print("Add the remainder back in to F")
    F.append(r)


The s polynomial is:


-2⋅x⋅y

The remainder is:


-2⋅x⋅y

Add the remainder back in to F


Now we compare $f_1$ with the newly added $f_4$:

In [414]:
s=s_polynomial(x**3-2*x*y, -2*x*y)
r = remainder_set_division(s, F)
r


      2 
-3⋅x⋅y  
────────
   2    

Ok , so this value is clearly wrong, let's look at the remainder values:

In [415]:
display(s)
for i in range(len(F)):
    print(div(s, F[i]))
# div(s, F[0])
# div(s, F[1])
# div(s, F[2])
# div(s, F[0])

      2
-2⋅x⋅y 

(0, -2*x*y**2)
(0, -2*x*y**2)
(0, -2*x*y**2)
(y, 0)


OK, so I think my strategy to calculate the remainder is incorrect because, clearly:

$$
-2xy^2 = 
0 \times (-2xy^2) +
0 \times (-2xy^2) +
0 \times (-2xy^2) +
y\times \left(-2xy\right)
(0)
$$


What if I tried multiplying?

In [ ]:
# the division algorithm, returns the quotient and remainder
# Taken from https://stackoverflow.com/questions/62002479/trying-to-write-the-division-algorithm-in-sympy-which-apparently-has-its-own-tru

def DivAlg(F,g):   
    n = len(F)
    Q = [0]*4
    r = f   # remainder

    while max([polynomial_degree(f) for f in F]) <= degree(r) and r!=0:
        if degree(g) <= degree(r)
    while (r != 0) & (degree(g) <= degree(r)):
        q = q + LT(r)/LT(g) 
        r = r - expand(( LT(r)/LT(g)) * g) # the expand command is necessary!

    return [q,r]

In [94]:
F

⎡ 3           2            2⎤
⎣x  - 2⋅x⋅y, x ⋅y + x - 2⋅y ⎦

In [22]:
# the division algorithm, returns the quotient and remainder
# Taken from https://stackoverflow.com/questions/62002479/trying-to-write-the-division-algorithm-in-sympy-which-apparently-has-its-own-tru

def DivAlg(f,g):   
    q = 0   # quotient
    r = f   # remainder

    while (r != 0) & (degree(g) <= degree(r)):
        q = q + LT(r)/LT(g) 
        r = r - expand(( LT(r)/LT(g)) * g) # the expand command is necessary!

    return [q,r]

In [24]:
DivAlg(x**2, x**3)

⎡    2⎤
⎣0, x ⎦

In [58]:
coeff   = [1, 3, 5, 7, 11, 13]
support = [x**3, x**2, x**3*y**2, x, y**2, y**4] 

terms   = [coeff[i]*support[i] for i in range(6)]
polynomial = sum(terms)

display(polynomial)

   3  2    3      2             4       2
5⋅x ⋅y  + x  + 3⋅x  + 7⋅x + 13⋅y  + 11⋅y 

In [59]:
# First get back the monomials from a polynomial
f = polynomial
monomials = [m/LC(m) for m in f.as_ordered_terms()]
monomials

⎡ 3  2   3   2      4   2⎤
⎣x ⋅y , x , x , x, y , y ⎦

In [75]:
m = monomials[0]
sets=[m.atoms(Pow) for m in monomials]
sets

⎡⎧ 3   2⎫  ⎧ 3⎫  ⎧ 2⎫         ⎧ 4⎫  ⎧ 2⎫⎤
⎢⎨x , y ⎬, ⎨x ⎬, ⎨x ⎬, set(), ⎨y ⎬, ⎨y ⎬⎥
⎣⎩      ⎭  ⎩  ⎭  ⎩  ⎭         ⎩  ⎭  ⎩  ⎭⎦

In [92]:
def monomial_degrees(polynomial):
    # Break the polynomial into a list of monomials
    monomials = [m/LC(m) for m in f.as_ordered_terms()]

    # Break the monomial into a list of sets of the variables that compose each monomial
    sets=[m.atoms(Pow) for m in monomials]

    # Break each variable into exponents and add up the degree of each monomial
    monomial_degrees = []
    for set in sets:
        degree = 0
        for var in set:
            exponent = var.as_base_exp()[1]
            degree += exponent
        monomial_degrees.append(degree)
    
    return monomial_degrees

def polynomial_degree(f):
    return max(monomial_degrees(f))

display(
    "Polynomial is",
    polynomial,
    "Degrees of each Monomial:",
    monomial_degrees(polynomial),
    "Degree of the Polynomial:",
    polynomial_degree(polynomial)
)

'Polynomial is'

   3  2    3      2             4       2
5⋅x ⋅y  + x  + 3⋅x  + 7⋅x + 13⋅y  + 11⋅y 

'Degrees of each Monomial:'

[5, 3, 2, 0, 4, 2]

'Degree of the Polynomial:'

5

In [73]:
expr=m
[z.as_base_exp()[1] for z in expr.atoms(Pow)]


[3, 2]

In [70]:
expr=m
min((z.as_base_exp()[1] for z in expr.atoms(sympy.Pow)), default=1)


NameError: name 'sympy' is not defined

In [41]:
q = polynomial.as_ordered_terms()
q
3+3

6

In [32]:
degree(x**2*y**3+x**5)

SyntaxError: positional argument follows keyword argument (<ipython-input-32-cf54514f01e7>, line 1)

In [19]:
 def remainder_set_division(s, F):
    """
    return remainder value after dividing s by all terms in F,
    For use in BuchBerger's Algorithm using an S-polynomial against the
    collection of polynomials

    Args:
        s (sympy): A sympy polynomial, typically an S-polynomial for 
        F (list, of sympy): The list of polynomials of concern
    """    
    r_list = [ div(s, F[i])[1] for i in range(len(F)) ]
    r = prod(r_list)**(1/len(r_list))
    return r.expand().doit()

r = remainder_set_division(s, F)    
r.doit()

    0.5
⎛ 4⎞   
⎝x ⎠   

In [427]:
div(s, prod(F))

(0, 0)

In [14]:
div(13*x*y, 4*y)

⎛13⋅x   ⎞
⎜────, 0⎟
⎝ 4     ⎠

In [417]:
divmod(s, F[0])

⎛⎢       2  ⎥                          ⎞
⎜⎢ -2⋅x⋅y   ⎥    ⎛⎛    2⎞      2      ⎞⎟
⎜⎢──────────⎥, x⋅⎝⎝-2⋅y ⎠ mod x  - 2⋅y⎠⎟
⎜⎢ 3        ⎥                          ⎟
⎝⎣x  - 2⋅x⋅y⎦                          ⎠

In [418]:
div(x*y, x)

(y, 0)

In [ ]:
 def remainder_set_division(s, F):
    """
    return remainder value after dividing s by all terms in F,
    For use in BuchBerger's Algorithm using an S-polynomial against the
    collection of polynomials

    Args:
        s (sympy): A sympy polynomial, typically an S-polynomial for 
        F (list, of sympy): The list of polynomials of concern
    """    
    r_list = [ div(s, F[i])[1] for i in range(len(F)) ]
    r = sum(r_list)/len(r_list)
    return r

r = remainder_set_division(s, F)    
r

In [419]:
# Calculate the S-Polynomial
s = s_polynomial(F[0], F[3])
print("The s polynomial is:")
display(s)
# calculate the remainder
r = remainder_set_division(s, F)    
print("The remainder is:")
display(r)

# Add the remainder back to F
if r!=0:
    print("Add the remainder back in to F")
    F.append(r)


The s polynomial is:


      2
-2⋅x⋅y 

The remainder is:


      2 
-3⋅x⋅y  
────────
   2    

Add the remainder back in to F


### Automate the process

In [420]:
F = [x**3 - 2*x*y, x**2*y + x - 2*y**2, -x**2]
display(F)


⎡ 3           2            2    2⎤
⎣x  - 2⋅x⋅y, x ⋅y + x - 2⋅y , -x ⎦

To return all possible pairs from a set of values in python, the itertools library can be used:

In [421]:
from itertools import tee



def pairwise(iterable): # NOTE https://docs.python.org/3/library/itertools.html
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    iterable.append(iterable[0]) # I have to add this on to make it work, it's not elegant but :shrug:
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)


list_of_vals = [1, 2, 3, 5, 7, 11, 13, 17, 19]
for pair in pairwise(F):
    display(pair)

⎛ 3           2            2⎞
⎝x  - 2⋅x⋅y, x ⋅y + x - 2⋅y ⎠

⎛ 2            2    2⎞
⎝x ⋅y + x - 2⋅y , -x ⎠

⎛  2   3        ⎞
⎝-x , x  - 2⋅x⋅y⎠

In [422]:
f=F[0]
g=F[1]
s = s_polynomial(f, g)
s
r_list = [ div(s, F[i])[1] for i in range(len(F)) ]

    # Add the final remainder
r = sum(r_list)/len(r_list)

In [423]:
F = F_0.copy()

# For every pair of polynomials in F
for pair in pairwise(F):
    # Print the pairs that are being used
    print("Input Polynomials are:")
    display(pair[0])
    display(pair[1])

    # Calculate the S Polynomial
    s = s_polynomial(pair[0], pair[1])

    # Calculate all the remainders
    r_list = [ div(s, F[i])[1] for i in range(len(F)) ]

    # Add the final remainder
    r = sum(r_list)/len(r_list)

    # If necessary add it back in
    if r != 0: 
        print("The remainder is:")
        display(r)
        F.append(r)
    else:
        print("Remainder is 0, this pair belongs to Groebner Basis")
# Print F, which should now be a Groebner Basis
print("Algorithm Completed! The Groebner Basis is:")
for poly in F:
    display(poly)

Input Polynomials are:


 3        
x  - 2⋅x⋅y

 2            2
x ⋅y + x - 2⋅y 

The remainder is:


  2
-x 

...

 2            2
x ⋅y + x - 2⋅y 

 3        
x  - 2⋅x⋅y

  2
-x 

   2
3⋅x 
────
 4  

-2⋅x⋅y

In [424]:
To test if this is indeed a Groebner Basis:

SyntaxError: invalid syntax (<ipython-input-424-6d2ae4931d5a>, line 1)

Now would be a good time to stop and look at implementing autoreduction.

### Old Work

In [17]:
def poly_prod(F):
    product_val = 1
    for poly in F:
        product_val *= poly
    return product_val

In [18]:
def remainder_set_div(s, F):
    r_list = [ div(s, F[i])[1] for i in range(len(F)) ]
    r = sum(r_list)
    return r


In [19]:
def buchberger_criterion(f,g, F):
    s = s_polynomial(f, g)
    r = remainder_set_div(s, F)
    return r

In [20]:
r = buchberger_criterion(F[0], F[1], F)
if r != 0:
    F.append(r)


In [21]:
F

⎡                                                              2              
⎢ 3           2            2    2   3           3          -4⋅x           2  1
⎢x  - 2⋅x⋅y, x ⋅y + x - 2⋅y , -x , x  - 2⋅x⋅y, x  - 2⋅x⋅y, ──────, x - 2⋅y , ─
⎣                                                            5                

           3               3       2       4      3      3      4       5     
2⋅x⋅y   4⋅y     3⋅x⋅y   5⋅y   4⋅x⋅y    25⋅y   -7⋅y    5⋅y   25⋅y   125⋅y      
───── + ────, - ───── - ────, ────── + ─────, ──────, ────, ─────, ──────, - 1
 7       7        2      12     3        81     3      99    648    702       

           ⎤
   2      4⎥
1⋅x  - 4⋅y ⎥
           ⎦

So we got back a remainder of $-x^2$, this isn't zero, so we throw it in the bag

We haven't tried every combination, so onto the next one:

In [22]:
r = buchberger_criterion(F[0], F[2], F)
if r != 0:
    F.append(r)


In [23]:
F

⎡                                                              2              
⎢ 3           2            2    2   3           3          -4⋅x           2  1
⎢x  - 2⋅x⋅y, x ⋅y + x - 2⋅y , -x , x  - 2⋅x⋅y, x  - 2⋅x⋅y, ──────, x - 2⋅y , ─
⎣                                                            5                

           3               3       2       4      3      3      4       5     
2⋅x⋅y   4⋅y     3⋅x⋅y   5⋅y   4⋅x⋅y    25⋅y   -7⋅y    5⋅y   25⋅y   125⋅y      
───── + ────, - ───── - ────, ────── + ─────, ──────, ────, ─────, ──────, - 1
 7       7        2      12     3        81     3      99    648    702       

                           3⎤
   2      4            25⋅y ⎥
1⋅x  - 4⋅y , -24⋅x⋅y - ─────⎥
                         9  ⎦

This gave a remainder of $-2xy$, so it goes in the bag

Now we try the next combination $f_1$ and $f_4$

In [24]:
r = buchberger_criterion(F[0], F[3], F)
if r != 0:
    F.append(r)
F

⎡                                                              2              
⎢ 3           2            2    2   3           3          -4⋅x           2  1
⎢x  - 2⋅x⋅y, x ⋅y + x - 2⋅y , -x , x  - 2⋅x⋅y, x  - 2⋅x⋅y, ──────, x - 2⋅y , ─
⎣                                                            5                

           3               3       2       4      3      3      4       5     
2⋅x⋅y   4⋅y     3⋅x⋅y   5⋅y   4⋅x⋅y    25⋅y   -7⋅y    5⋅y   25⋅y   125⋅y      
───── + ────, - ───── - ────, ────── + ─────, ──────, ────, ─────, ──────, - 1
 7       7        2      12     3        81     3      99    648    702       

                           3⎤
   2      4            25⋅y ⎥
1⋅x  - 4⋅y , -24⋅x⋅y - ─────⎥
                         9  ⎦

So the problem at this step is that diving by the product of F is not what we want

In [25]:
print(div(-2*x*y**2, F[1]))
print(div(-2*x*y**2, F[2]))
print(div(-2*x*y**2, F[3]))
print(div(-2*x*y**2, F[4]))

(0, -2*x*y**2)
(0, -2*x*y**2)
(0, -2*x*y**2)
(0, -2*x*y**2)
